In [ ]:
! pip install wandb

In [ ]:
import wandb

wandb.login()

wandb.init(project="uber-mlops")


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: buck2305 (uber-mlops). Use `wandb login --relogin` to force relogin


In [ ]:
import pandas as pd
import numpy as np

uber = pd.read_csv('uber.csv')
uber.drop('Unnamed: 0', axis=1, inplace=True)

uber['pickup_year']=pd.DatetimeIndex(uber['pickup_datetime']).year
uber['pickup_month']=pd.DatetimeIndex(uber['pickup_datetime']).month
uber['pickup_day']=pd.DatetimeIndex(uber['pickup_datetime']).day
uber['pickup_hour']=pd.DatetimeIndex(uber['pickup_datetime']).hour
uber['pickup_minute']=pd.DatetimeIndex(uber['pickup_datetime']).minute
uber['pickup_second']=pd.DatetimeIndex(uber['pickup_datetime']).second

# cycling encoding for pickup_month, pickup_day, pickup_hour, pickup_minute, pickup_second

uber['pickup_month_sin'] = np.sin(uber['pickup_month']*(2.*np.pi/12))
uber['pickup_month_cos'] = np.cos(uber['pickup_month']*(2.*np.pi/12))
uber['pickup_day_sin'] = np.sin(uber['pickup_day']*(2.*np.pi/31))
uber['pickup_day_cos'] = np.cos(uber['pickup_day']*(2.*np.pi/31))
uber['pickup_hour_sin'] = np.sin(uber['pickup_hour']*(2.*np.pi/24))
uber['pickup_hour_cos'] = np.cos(uber['pickup_hour']*(2.*np.pi/24))
uber['pickup_minute_sin'] = np.sin(uber['pickup_minute']*(2.*np.pi/60))
uber['pickup_minute_cos'] = np.cos(uber['pickup_minute']*(2.*np.pi/60))
uber['pickup_second_sin'] = np.sin(uber['pickup_second']*(2.*np.pi/60))
uber['pickup_second_cos'] = np.cos(uber['pickup_second']*(2.*np.pi/60))

uber.drop('pickup_month', axis=1, inplace=True)
uber.drop('pickup_day', axis=1, inplace=True)
uber.drop('pickup_hour', axis=1, inplace=True)
uber.drop('pickup_minute', axis=1, inplace=True)
uber.drop('pickup_second', axis=1, inplace=True)

uber.drop('pickup_datetime', axis=1, inplace=True)
uber.drop('key', axis=1, inplace=True)
uber.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month_sin,pickup_month_cos,pickup_day_sin,pickup_day_cos,pickup_hour_sin,pickup_hour_cos,pickup_minute_sin,pickup_minute_cos,pickup_second_sin,pickup_second_cos
0,7.5,-73.999817,40.738354,-73.999512,40.723217,1,2015,5.000000e-01,-0.866025,0.988468,0.151428,-0.965926,0.258819,-0.743145,6.691306e-01,0.587785,0.809017
1,7.7,-73.994355,40.728225,-73.994710,40.750325,1,2009,-5.000000e-01,-0.866025,-0.299363,-0.954139,-0.866025,0.500000,0.406737,9.135455e-01,-0.406737,0.913545
2,12.9,-74.005043,40.740770,-73.962565,40.772647,1,2009,-8.660254e-01,-0.500000,-0.988468,0.151428,-0.707107,0.707107,-1.000000,-1.836970e-16,0.000000,1.000000
3,5.3,-73.976124,40.790844,-73.965316,40.803349,3,2009,1.224647e-16,-1.000000,-0.848644,0.528964,0.866025,-0.500000,0.743145,-6.691306e-01,0.809017,-0.587785
4,16.0,-73.925023,40.744085,-73.973082,40.761247,5,2014,-8.660254e-01,-0.500000,-0.571268,0.820763,-0.965926,-0.258819,-0.978148,2.079117e-01,0.000000,1.000000


In [ ]:
uber[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']].describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
count,200000.000000,200000.000000,199999.000000,199999.000000
mean,-72.527638,39.935885,-72.525292,39.923890
std,11.437787,7.720539,13.117408,6.794829
min,-1340.648410,-74.015515,-3356.666300,-881.985513
25%,-73.992065,40.734796,-73.991407,40.733823
50%,-73.981823,40.752592,-73.980093,40.753042
75%,-73.967154,40.767158,-73.963658,40.768001
max,57.418457,1644.421482,1153.572603,872.697628


In [ ]:
uber.drop(uber[uber['pickup_longitude']>90].index, inplace=True)
uber.drop(uber[uber['pickup_latitude']>90].index, inplace=True)
uber.drop(uber[uber['dropoff_longitude']>90].index, inplace=True)
uber.drop(uber[uber['dropoff_latitude']>90].index, inplace=True)

uber.drop(uber[uber['pickup_longitude']<-90].index, inplace=True)
uber.drop(uber[uber['pickup_latitude']<-90].index, inplace=True)
uber.drop(uber[uber['dropoff_longitude']<-90].index, inplace=True)
uber.drop(uber[uber['dropoff_latitude']<-90].index, inplace=True)

uber.drop(uber[uber['passenger_count'] > 5].index, axis=0, inplace = True)
uber.drop(uber[uber['passenger_count'] == 0].index, axis=0, inplace = True)
uber.drop(uber[uber['fare_amount'] < 2.5].index, axis=0, inplace = True)

uber.dropna(inplace=True)

uber.shape

(194981, 17)

In [ ]:
uber[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']].describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
count,194981.000000,194981.000000,194981.000000,194981.000000
mean,-72.503114,39.920694,-72.515315,39.926626
std,10.434251,6.087745,10.388957,6.069382
min,-89.933333,-74.015515,-75.458979,-74.015750
25%,-73.992058,40.734779,-73.991400,40.733862
50%,-73.981828,40.752568,-73.980090,40.753045
75%,-73.967167,40.767145,-73.963662,40.768003
max,40.808425,48.018760,40.831932,45.031598


In [ ]:
uber['pickup_coords'] = list(zip(uber.pickup_latitude, uber.pickup_longitude))
uber['dropoff_coords'] = list(zip(uber.dropoff_latitude, uber.dropoff_longitude))

uber.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month_sin,pickup_month_cos,pickup_day_sin,pickup_day_cos,pickup_hour_sin,pickup_hour_cos,pickup_minute_sin,pickup_minute_cos,pickup_second_sin,pickup_second_cos,pickup_coords,dropoff_coords
0,7.5,-73.999817,40.738354,-73.999512,40.723217,1,2015,5.000000e-01,-0.866025,0.988468,0.151428,-0.965926,0.258819,-0.743145,6.691306e-01,0.587785,0.809017,"(40.73835372924805, -73.99981689453125)","(40.72321701049805, -73.99951171875)"
1,7.7,-73.994355,40.728225,-73.994710,40.750325,1,2009,-5.000000e-01,-0.866025,-0.299363,-0.954139,-0.866025,0.500000,0.406737,9.135455e-01,-0.406737,0.913545,"(40.728225, -73.994355)","(40.750325, -73.99471)"
2,12.9,-74.005043,40.740770,-73.962565,40.772647,1,2009,-8.660254e-01,-0.500000,-0.988468,0.151428,-0.707107,0.707107,-1.000000,-1.836970e-16,0.000000,1.000000,"(40.74077, -74.005043)","(40.772647, -73.962565)"
3,5.3,-73.976124,40.790844,-73.965316,40.803349,3,2009,1.224647e-16,-1.000000,-0.848644,0.528964,0.866025,-0.500000,0.743145,-6.691306e-01,0.809017,-0.587785,"(40.790844, -73.976124)","(40.803349, -73.965316)"
4,16.0,-73.925023,40.744085,-73.973082,40.761247,5,2014,-8.660254e-01,-0.500000,-0.571268,0.820763,-0.965926,-0.258819,-0.978148,2.079117e-01,0.000000,1.000000,"(40.744085, -73.925023)","(40.761247, -73.97308199999999)"


In [ ]:
import geopy.distance

def distance(row):
    return geopy.distance.geodesic(row['pickup_coords'], row['dropoff_coords']).miles

uber['distance'] = uber.apply(distance, axis=1)
uber.drop(uber[uber['distance'] > 130].index, axis=0, inplace = True)
uber.drop(uber[uber['distance'] == 0].index, axis=0, inplace = True)


uber.drop('pickup_coords', axis=1, inplace=True)
uber.drop('dropoff_coords', axis=1, inplace=True)

uber.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month_sin,pickup_month_cos,pickup_day_sin,pickup_day_cos,pickup_hour_sin,pickup_hour_cos,pickup_minute_sin,pickup_minute_cos,pickup_second_sin,pickup_second_cos,distance
0,7.5,-73.999817,40.738354,-73.999512,40.723217,1,2015,5.000000e-01,-0.866025,0.988468,0.151428,-0.965926,0.258819,-0.743145,6.691306e-01,0.587785,0.809017,1.044594
1,7.7,-73.994355,40.728225,-73.994710,40.750325,1,2009,-5.000000e-01,-0.866025,-0.299363,-0.954139,-0.866025,0.500000,0.406737,9.135455e-01,-0.406737,0.913545,1.525071
2,12.9,-74.005043,40.740770,-73.962565,40.772647,1,2009,-8.660254e-01,-0.500000,-0.988468,0.151428,-0.707107,0.707107,-1.000000,-1.836970e-16,0.000000,1.000000,3.131464
3,5.3,-73.976124,40.790844,-73.965316,40.803349,3,2009,1.224647e-16,-1.000000,-0.848644,0.528964,0.866025,-0.500000,0.743145,-6.691306e-01,0.809017,-0.587785,1.032372
4,16.0,-73.925023,40.744085,-73.973082,40.761247,5,2014,-8.660254e-01,-0.500000,-0.571268,0.820763,-0.965926,-0.258819,-0.978148,2.079117e-01,0.000000,1.000000,2.786061


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X = uber.drop(['fare_amount','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'], axis=1)
y = uber['fare_amount']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
# apply linear regression with wandb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

wandb.log({'mse': mse, 'r2': r2})

In [ ]:
# apply random forest with wandb
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth=wandb.config.max_depth, max_features=wandb.config.max_features, n_estimators=wandb.config.n_estimators)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)



In [ ]:
# apply random forest with hyperopt and wandb

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

space = {
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'max_features': hp.quniform('max_features', 2, 5, 1),
    'n_estimators': hp.quniform('n_estimators', 50, 150, 10)
}

def objective(space):
    rf = RandomForestRegressor(max_depth=int(space['max_depth']), max_features=int(space['max_features']), n_estimators=int(space['n_estimators']))
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    wandb.log({'max_depth': space['max_depth'], 'max_features': space['max_features'], 'n_estimators': space['n_estimators'], 'mse': mse})
    return {'loss': mse, 'status': STATUS_OK}

trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)
wandb.finish()
print(best)

100%|██████████| 10/10 [02:22<00:00, 14.20s/trial, best loss: 25.433122498675782]
{'max_depth': 10.0, 'max_features': 5.0, 'n_estimators': 110.0}


In [ ]:
wandb.finish()

max_depth,█▆▃▄▁▃▄▅▄▆
max_features,█▅▁██▅▁▁▅▁
mse,▅▁▄▂█▃▂▃▃▆█▇▁▂▆▃▇▅▅▄▃▃
n_estimators,▅▅▄▃▁▃▃▄█▂
r2,▁█
max_depth,8.0
max_features,3.0
mse,32.84016
n_estimators,80.0
r2,0.753


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor

space = {
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'max_features': hp.quniform('max_features', 2, 5, 1),
    'n_estimators': hp.quniform('n_estimators', 50, 150, 10)
}

# change run name
wandb.init(project="uber-mlops", name="gradient-boosting")

def objective(space):
    gbr = GradientBoostingRegressor(max_depth=int(space['max_depth']), max_features=int(space['max_features']), n_estimators=int(space['n_estimators']))
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    # log the current status of the run
    wandb.log({'max_depth': space['max_depth'], 'max_features': space['max_features'], 'n_estimators': space['n_estimators'], 'mse': mse})
    return {'loss': mse, 'status': STATUS_OK}

trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

wandb.finish()

print(best)

100%|██████████| 10/10 [02:59<00:00, 17.98s/trial, best loss: 24.34542129175292]


max_depth,▁▅▄▂▃█▅▄▆▄
max_features,▃▃▆▆▃▁▃▁▃█
mse,█▂▃▂▄▄▁▄▃▂
n_estimators,▅▅▁▆▄▆▅█▇▆
max_depth,5.0
max_features,5.0
mse,24.6286
n_estimators,110.0


{'max_depth': 7.0, 'max_features': 3.0, 'n_estimators': 100.0}


In [ ]:
from xgboost import XGBRegressor
import os
os.environ["WANDB_SILENT"] = "true"

# apply hyperopt with xgboost regressor and wandb

space = {
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'max_features': hp.quniform('max_features', 2, 5, 1),
    'n_estimators': hp.quniform('n_estimators', 50, 150, 10)
}

def objective(space):
    wandb.init(project="uber-mlops", name="xgboost"+str(space['max_depth'])+str(space['max_features'])+str(space['n_estimators']))
    xgb = XGBRegressor(max_depth=int(space['max_depth']), max_features=int(space['max_features']), n_estimators=int(space['n_estimators']))
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    # log the current status of the run
    wandb.log({'max_depth': space['max_depth'], 'max_features': space['max_features'], 'n_estimators': space['n_estimators'], 'mse': mse})
    wandb.finish()
    return {'loss': mse, 'status': STATUS_OK}

trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

print(best)